In [1]:
import pandas as pd

import numpy as np
import matplotlib.pyplot as plt

In [2]:
dataset = pd.read_csv('Social_Network_Ads.csv')

In [3]:
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [4]:
dataset = pd.get_dummies(dataset,drop_first = True)

In [5]:
dataset

,User ID,Age,EstimatedSalary,Purchased,Gender_Male
0,15624510,19,19000,0,1
1,15810944,35,20000,0,1
2,15668575,26,43000,0,0
3,15603246,27,57000,0,0
4,15804002,19,76000,0,1
...,...,...,...,...,...
395,15691863,46,41000,1,0
396,15706071,51,23000,1,1
397,15654296,50,20000,1,0
398,15755018,36,33000,0,1


In [6]:
dataset.columns

Index(['User ID', 'Age', 'EstimatedSalary', 'Purchased', 'Gender_Male'], dtype='object')

In [7]:
dataset = dataset.drop('User ID' , axis = 1)

In [8]:
dataset

,Age,EstimatedSalary,Purchased,Gender_Male
0,19,19000,0,1
1,35,20000,0,1
2,26,43000,0,0
3,27,57000,0,0
4,19,76000,0,1
...,...,...,...,...
395,46,41000,1,0
396,51,23000,1,1
397,50,20000,1,0
398,36,33000,0,1


In [9]:
dataset['Purchased'].value_counts()

0    257
1    143
Name: Purchased, dtype: int64

In [10]:
indep = dataset[['Age', 'EstimatedSalary', 'Gender_Male']]

dep = dataset['Purchased']

In [11]:
indep.shape

(400, 3)

In [12]:
indep

,Age,EstimatedSalary,Gender_Male
0,19,19000,1
1,35,20000,1
2,26,43000,0
3,27,57000,0
4,19,76000,1
...,...,...,...
395,46,41000,0
396,51,23000,1
397,50,20000,0
398,36,33000,1


In [13]:
dep

0      0
1      0
2      0
3      0
4      0
      ..
395    1
396    1
397    1
398    0
399    1
Name: Purchased, Length: 400, dtype: int64

In [14]:
from sklearn.model_selection import train_test_split

X_train,X_test,Y_train,Y_test = train_test_split(indep,dep, test_size = 0.30, random_state = 0)

In [15]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [16]:
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import GridSearchCV

param_grid = {'penalty' : ['l1','l2','elasticnet'],
              'solver': ['liblinear', 'newton-cg', 'newton-cholesky', 'sag']
              }
grid = GridSearchCV(LogisticRegression(),param_grid,refit = 'True', verbose = 3,n_jobs = -1, scoring = 'f1_weighted')
#fiiting the model for grid
grid.fit(X_train,Y_train)



Fitting 5 folds for each of 12 candidates, totalling 60 fits


C:\Users\Admin\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
40 fits failed out of a total of 60.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\Admin\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Admin\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py", line 1461, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "C:\Users\Admin\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py", line 449, in _check_solver
    % (solver, penalty)
Val

GridSearchCV(estimator=LogisticRegression(), n_jobs=-1,
             param_grid={'penalty': ['l1', 'l2', 'elasticnet'],
                         'solver': ['liblinear', 'newton-cg', 'newton-cholesky',
                                    'sag']},
             refit='True', scoring='f1_weighted', verbose=3)

In [17]:
#print best parameter after tuning 
#print(grid.best_params_)

re = grid.cv_results_

grid_predictions = grid.predict(X_test)



from sklearn.metrics import confusion_matrix
cm = confusion_matrix (Y_test,grid_predictions)

#print classification report

from sklearn.metrics import classification_report

clf_report  = classification_report(Y_test,grid_predictions)

In [18]:
from sklearn.metrics import f1_score

f1_macro = f1_score(Y_test,grid_predictions,average = 'weighted')

print('The R_score vlaue for best parameter {}:'.format(grid.best_params_),f1_score)

The R_score vlaue for best parameter {'penalty': 'l1', 'solver': 'liblinear'}: <function f1_score at 0x00000235BBF3AA68>


In [19]:
print (cm)

[[74  5]
 [ 8 33]]


In [20]:
print (clf_report)

              precision    recall  f1-score   support

           0       0.90      0.94      0.92        79
           1       0.87      0.80      0.84        41

    accuracy                           0.89       120
   macro avg       0.89      0.87      0.88       120
weighted avg       0.89      0.89      0.89       120



In [21]:
from sklearn.metrics import roc_auc_score

roc_auc_score(Y_test,grid.predict_proba(X_test)[:,-1])

0.9493670886075949

In [22]:
table = pd.DataFrame.from_dict(re)

In [23]:
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_penalty,param_solver,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.039578,0.000799,0.024786,0.007906,l1,liblinear,"{'penalty': 'l1', 'solver': 'liblinear'}",0.835985,0.802399,0.644599,0.927778,0.909115,0.823975,0.100806,1
1,0.010395,0.007655,0.000000,0.000000,l1,newton-cg,"{'penalty': 'l1', 'solver': 'newton-cg'}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,5
2,0.002400,0.000491,0.000000,0.000000,l1,newton-cholesky,"{'penalty': 'l1', 'solver': 'newton-cholesky'}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,6
3,0.003198,0.000979,0.000000,0.000000,l1,sag,"{'penalty': 'l1', 'solver': 'sag'}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,7
4,0.146705,0.066999,0.030783,0.013782,l2,liblinear,"{'penalty': 'l2', 'solver': 'liblinear'}",0.835985,0.802399,0.644599,0.927778,0.890114,0.820175,0.097839,2
5,0.055170,0.020865,0.011392,0.002244,l2,newton-cg,"{'penalty': 'l2', 'solver': 'newton-cg'}",0.835985,0.802399,0.644599,0.927778,0.890114,0.820175,0.097839,2
6,0.002999,0.001095,0.000000,0.000000,l2,newton-cholesky,"{'penalty': 'l2', 'solver': 'newton-cholesky'}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,8
7,0.074380,0.032250,0.008795,0.000977,l2,sag,"{'penalty': 'l2', 'solver': 'sag'}",0.835985,0.802399,0.644599,0.927778,0.890114,0.820175,0.097839,2
8,0.001799,0.000400,0.000000,0.000000,elasticnet,liblinear,"{'penalty': 'elasticnet', 'solver': 'liblinear'}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,9
9,0.002800,0.000749,0.000000,0.000000,elasticnet,newton-cg,"{'penalty': 'elasticnet', 'solver': 'newton-cg'}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,10
